In [27]:
import sys
sys.path.insert(0, '../')

In [2]:
import json
import pandas as pd
import numpy as np
from src.agent import Pipeline, PipelineAgentConfig, IOAgent, AgentContext, registry
from src.llm_client import LLMClient
from src.schema.llm_config import LLMConfig
from src.utils.data import SyntheticEmotionDataset

In [3]:
with open("../configs/gpt_4o_mini_config.json", "r") as f:
    config_dct = json.load(f)
llm_config = LLMConfig.from_dict(config_dct)
llm_client = LLMClient(llm_config)

In [5]:
dset = SyntheticEmotionDataset("../data/synthetic_dialogues/v2/dialogues.json", "../data/synthetic_dialogues/v2/scenarios.json")

In [7]:
gt_emotion = [dset[i].emotion.value for i in range(200)]

In [8]:
with open("../data/baseline_erc_bert.json", "r") as f:
    data = json.load(f)["predictions"]

In [12]:
def get_argmax(res):
    keys = list(res.keys())
    values = list(res.values())
    index_max = np.argmax(values)
    return keys[index_max]

In [13]:
baseline_predictions = [get_argmax(item['prediction']) for item in  data[:200]]

In [15]:
with open("../data/inside_out_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]

In [19]:
predictions_inside_out = [item['prediction'].split('; ')[0].lower() for item in data]

In [39]:
with open("../data/baseline_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
baseline_predicitons = [i['predicted'].split(';')[0].lower() for i in data]
with open("../data/inside_out_v1_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
inside_out_predictions = [i['predicted'].split(';')[0].lower() for i in data]
with open("../data/self_consistency_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
self_consistency_predictions = [i['predicted'].split(';')[0].lower() for i in data]

In [ ]:
baseline_predicitons

In [22]:
def accuracy(gt, pred):
    mask = [u == v for u, v in zip(gt, pred)]
    return np.array(mask).mean()

In [49]:
results = {
    "inside-out": accuracy(gt_emotion, inside_out_predictions),
    "baseline": accuracy(gt_emotion, baseline_predicitons), 
    "self consistency": accuracy(gt_emotion, self_consistency_predictions)
}
results_df = pd.DataFrame([results])
results_df

In [23]:
accuracy(gt_emotion, predictions_inside_out)

0.735

In [24]:
accuracy(gt_emotion, baseline_predictions)

0.645

In [ ]:
bert baseline: 0.735, inside out: 0.735